# 帯域制御装置 トラヒック統計分析
**目的:** 新規帯域制御装置の導入可否判断のためのトラヒック統計情報の可視化

このノートブックは `src/` 配下のモジュールを呼び出す薄いラッパーです。
ロジック本体は以下のモジュールにあります:

| モジュール | 役割 |
|-----------|------|
| `src/config.py` | パス・デフォルト設定の一元管理 |
| `src/calc_traffic.py` | Byte⇔Mbps変換などの計算 |
| `src/sample_data.py` | サンプルデータ生成 |
| `src/merge_csv.py` | 3種CSV.gz → 統合CSV マージ |
| `src/graphs.py` | 5種グラフ描画関数 |
| `main.py` | CLIエントリーポイント（`python main.py --all`） |

## セル構成
| セル | 内容 |
|------|------|
| セットアップ | モジュール読み込み |
| サンプルデータ生成 | `sample_data.generate_sample_data()` |
| CSV統合 | `merge_csv.merge_traffic_csv()` |
| 統合CSV読み込み | `pd.read_csv()` → `df` |
| グラフ1 | `graphs.plot_graph1()` |
| グラフ2 | `graphs.plot_graph2()` |
| グラフ3 | `graphs.plot_graph3()` |
| グラフ4 | `graphs.plot_graph4()` |
| グラフ5 | `graphs.plot_graph5()` |

In [1]:
"""
セットアップ: モジュール読み込みとディレクトリ確認
"""
import os
import sys
import pandas as pd

# ノートブックの場所を基準にsrc/をインポートできるようにする
# （Jupyterのcwdがノートブックと異なる場合に対応）
_NOTEBOOK_DIR = os.path.dirname(os.path.abspath(
    globals().get("__vsc_ipynb_file__", "")  # VSCode
)) if "__vsc_ipynb_file__" in globals() else os.getcwd()

# notebook/bandwidth_analysis/ を探す
for candidate in [
    os.getcwd(),                                                  # カレントディレクトリ
    os.path.join(os.getcwd(), "notebook", "bandwidth_analysis"),  # Jupyterルートから
    os.path.dirname(os.path.abspath("__file__")),                 # ファイル基準
]:
    if os.path.isdir(os.path.join(candidate, "src")):
        _NOTEBOOK_DIR = candidate
        break

if _NOTEBOOK_DIR not in sys.path:
    sys.path.insert(0, _NOTEBOOK_DIR)
    os.chdir(_NOTEBOOK_DIR)

# パス定数・デフォルトパラメータ読み込み
from src.config import DATA_DIR, OUTPUT_DIR, ensure_dirs
from src.config import (
    NEW_TRAFFIC_FILENAME, CURRENT_TRAFFIC_FILENAME,
    BANDWIDTH_LIMIT_FILENAME, MERGED_CSV_FILENAME,
)

# モジュール読み込み
from src.sample_data import generate_sample_data
from src.merge_csv import merge_traffic_csv
from src.graphs import plot_graph1, plot_graph2, plot_graph3, plot_graph4

# ディレクトリ作成
ensure_dirs()

# 実行結果の通知
print("セットアップ完了")
print(f"  CWD:        {os.getcwd()}")
print(f"  DATA_DIR:   {DATA_DIR}")
print(f"  OUTPUT_DIR: {OUTPUT_DIR}")

セットアップ完了
  CWD:        /home/asama/notebook/bandwidth_analysis_test
  DATA_DIR:   /home/asama/notebook/bandwidth_analysis_test/data
  OUTPUT_DIR: /home/asama/notebook/bandwidth_analysis_test/output

In [2]:
"""
サンプルデータ生成（14日分）
============================
本番データがある場合はこのセルをスキップしてください。
"""
result = generate_sample_data(data_dir=DATA_DIR)

# 実行結果の通知
print(f"サンプルデータ生成完了:")
print(f"  IDs: {result['ids']}")
print(f"  新規: {result['num_rows_new']} rows")
print(f"  現行: {result['num_rows_current']} rows")
print(f"  limit: {result['num_rows_limit']} rows")

サンプルデータ生成完了:
  IDs: ['AA00-00-2015', 'BB01-01-2015', 'CC02-02-2015']
  新規: 12096 rows
  現行: 12096 rows
  limit: 3024 rows

In [3]:
"""
CSV統合: 3種CSV.gz → 統合CSV (merged_traffic.csv)
"""
new_path = os.path.join(DATA_DIR, NEW_TRAFFIC_FILENAME)
cur_path = os.path.join(DATA_DIR, CURRENT_TRAFFIC_FILENAME)
lim_path = os.path.join(DATA_DIR, BANDWIDTH_LIMIT_FILENAME)
out_path = os.path.join(DATA_DIR, MERGED_CSV_FILENAME)

df = merge_traffic_csv(new_path, cur_path, lim_path, out_path)

print(f"統合CSV生成完了:")
print(f"  レコード数: {len(df)}")
print(f"  カラム: {list(df.columns)}")
print(f"  ID一覧: {df['id'].unique().tolist()}")
print(f"  期間: {df['timestamp'].min()} ~ {df['timestamp'].max()}")
print(f"  出力先: {out_path} ({os.path.getsize(out_path):,} bytes)")

統合CSV生成完了:
  レコード数: 12096
  カラム: ['timestamp', 'id', 'limit_group', 'poi_code', 'new_pre_control_mbps_in', 'limit_mbps_in', 'new_volume_mbps_in', 'new_volume_mbps_out', 'new_dropped_mbps_in', 'cur_volume_mbps_in', 'cur_volume_mbps_out', 'new_volume_bytes_in', 'new_volume_bytes_out', 'cur_volume_bytes_in', 'cur_volume_bytes_out', 'limit_kbps_in', 'new_dropped_bytes_in', 'new_dropped_packets_in']
  ID一覧: ['AA00-00-2015', 'BB01-01-2015', 'CC02-02-2015']
  期間: 2025-01-15 00:00:00 ~ 2025-01-28 23:55:00
  出力先: /home/asama/notebook/bandwidth_analysis_test/data/merged_traffic.csv (1,880,565 bytes)

In [4]:
"""
統合CSVの読み込み
=================
既に統合CSVが存在する場合、前のセルをスキップして
このセルから実行できます。
"""
csv_path = os.path.join(DATA_DIR, MERGED_CSV_FILENAME)
df = pd.read_csv(csv_path, parse_dates=["timestamp"])
all_ids = df["id"].unique()

print(f"統合CSV読み込み完了: {len(df)} rows, IDs: {all_ids.tolist()}")

統合CSV読み込み完了: 12096 rows, IDs: ['AA00-00-2015', 'BB01-01-2015', 'CC02-02-2015']

In [5]:
"""
グラフ1: 新規 vs 現行 トラヒック比較
=====================================
パラメータを変更して再実行できます。
"""
# === パラメータ（変更可能） ===
TARGET_DATE = "2025-01-15"        # 対象日
TARGET_IDS = None                  # None=全ID, またはリストで指定 例: ["AA00-00-2015"]

saved = plot_graph1(df, TARGET_DATE, OUTPUT_DIR, TARGET_IDS)
for f in saved:
    print(f"  Saved: {os.path.basename(f)}")

  Saved: graph1_AA00-00-2015_2025-01-15.png
  Saved: graph1_BB01-01-2015_2025-01-15.png
  Saved: graph1_CC02-02-2015_2025-01-15.png

In [6]:
"""
グラフ2: 帯域制御時の積み上げ棒グラフ + limit
===============================================
"""
TARGET_DATE = "2025-01-15"
TARGET_IDS = None

saved = plot_graph2(df, TARGET_DATE, OUTPUT_DIR, TARGET_IDS)
for f in saved:
    print(f"  Saved: {os.path.basename(f)}")

  Saved: graph2_AA00-00-2015_2025-01-15.png
  Saved: graph2_BB01-01-2015_2025-01-15.png
  Saved: graph2_CC02-02-2015_2025-01-15.png

In [7]:
"""
グラフ3: 帯域制御精度 箱ひげ図（全ID横並び）
=============================================
"""
TARGET_DATE = "2025-01-15"

fpath = plot_graph3(df, TARGET_DATE, OUTPUT_DIR)
print(f"  Saved: {os.path.basename(fpath)}")

  Saved: graph3_boxplot_2025-01-15.png

In [8]:
"""
グラフ4: トラヒック量 vs 精度散布図（最大14日間, ID毎）
============================================
"""
START_DATE = "2025-01-15"
END_DATE = "2025-01-28"
TARGET_IDS = None

saved = plot_graph4(df, START_DATE, END_DATE, OUTPUT_DIR, TARGET_IDS)
for f in saved:
    print(f"  Saved: {os.path.basename(f)}")

  Saved: graph4_scatter_AA00-00-2015.png
  Saved: graph4_scatter_BB01-01-2015.png
  Saved: graph4_scatter_CC02-02-2015.png

## 出力ファイル一覧確認

In [10]:
# 出力ファイル一覧
print("=== 出力ファイル ===")
for f in sorted(os.listdir(OUTPUT_DIR)):
    fpath = os.path.join(OUTPUT_DIR, f)
    if os.path.isfile(fpath):
        size = os.path.getsize(fpath)
        print(f"  {f}  ({size:,} bytes)")
print(f"\n合計: {len([f for f in os.listdir(OUTPUT_DIR) if os.path.isfile(os.path.join(OUTPUT_DIR, f))])} files")

=== 出力ファイル ===
  graph1_AA00-00-2015_2025-01-15.png  (201,489 bytes)
  graph1_BB01-01-2015_2025-01-15.png  (218,457 bytes)
  graph1_CC02-02-2015_2025-01-15.png  (208,045 bytes)
  graph2_AA00-00-2015_2025-01-15.png  (103,632 bytes)
  graph2_BB01-01-2015_2025-01-15.png  (116,024 bytes)
  graph2_CC02-02-2015_2025-01-15.png  (115,188 bytes)
  graph3_boxplot_2025-01-15.png  (66,300 bytes)
  graph4_scatter_AA00-00-2015.png  (85,302 bytes)
  graph4_scatter_BB01-01-2015.png  (132,100 bytes)
  graph4_scatter_CC02-02-2015.png  (117,685 bytes)
  graph5_heatmap_AA00-00-2015.png  (78,272 bytes)
  graph5_heatmap_BB01-01-2015.png  (81,029 bytes)
  graph5_heatmap_CC02-02-2015.png  (80,189 bytes)

合計: 13 files